# A simple PageRank implementation

In [1]:
import json
import numpy as np

In [2]:
def read_graph(filename):
    # Open the file specified by 'filename' in read mode
    with open(filename, 'r') as f:
        # Load the graph data from the JSON file into the variable 'g'
        g = json.load(f)
        # Return the graph data; no further processing is required as it's already in the desired format
        return g

In [3]:
read_graph("data/example.json")

{'a': ['b', 'c', 'd'],
 'b': ['e', 'f'],
 'c': ['e'],
 'd': ['a'],
 'e': ['d'],
 'f': ['a']}

In [4]:
def compute_R(graph):
    # Calculate the number of nodes in the graph
    n = len(graph.keys())

    # we create a dictionary mapping from each node (key in the graph) to a unique index (position in the matrix)
    key_to_pos = dict(zip(graph.keys(), range(0,n)))

    # Initialize an nxn zero matrix, where n is the number of nodes in the graph
    R = np.zeros((n,n))

    # Iterate over each node in the graph
    for i, source in enumerate(graph.keys()):
        # Calculate the out-degree of the current node ('source'), which is the lentgh of its adjacent list 
        out_deg = len(graph[source])

        # Iterate over each destination node that 'source' is connected to
        for dest in graph[source]:
            # Find the matrix index corresponding to the destination node
            j = key_to_pos[dest]

            # Update the matrix entry to represent the edge weight from 'source' to 'dest'
            # Here, it is set as 1 divided by the out-degree of 'source'
            R[i][j] = 1/out_deg

    # Return the matrix representing the graph
    return R

In [5]:
def PageRank_iteration(x, R, J, alpha):
    # Determine the size of the vector x (number of nodes in the graph)
    n = len(x)

    # Create a column vector of ones with the same length as x
    one = np.asmatrix(np.ones(n)).T

    # Calculate the transition probability matrix P
    # P is a weighted combination of a random jump matrix J and the graph's adjacency matrix R
    # 'alpha' is the damping factor: it balances between the random jump and following links in R
    P = (alpha * one * J + (1 - alpha) * R)

    # Perform the PageRank iteration: multiply the current rank vector x with the transition matrix P
    x_prime = x * P

    # Return the updated rank vector
    return x_prime


In [6]:
def compute_PageRank(graph, alpha, epsilon):
    # Get the number of nodes in the graph
    n = len(graph.keys())

    # Compute the transition matrix R for the graph without considering teleportation
    R = compute_R(graph)

    # Initialize the jump vector J, a uniform distribution vector where each entry is 1/n
    J = np.ones(n)/n

    # Initialize the PageRank vector x with a uniform distribution (each entry is 1/n)
    x = np.ones(n)/n
    # Alternative initialization for x: a random stochastic vector
    # x = np.random.rand(n)
    # x = x/x.sum()

    # Initialize the error measure to infinity for the while loop condition
    err = np.inf

    # Iterate until the sum of the absolute differences between new and old x falls below epsilon
    while (err > epsilon):
        # Perform a PageRank iteration to get the new rank vector
        x_new = PageRank_iteration(x, R, J, alpha)

        # Update the error measure as the sum of absolute differences between new and old x
        err = (abs(x_new - x)).sum()

        # Debugging: print the current error value
        print(err)

        # Update the rank vector for the next iteration
        x = x_new

    # Print the final PageRank scores for each node in the graph
    print("PageRank scores:")
    for i, k in enumerate(graph.keys()):
        print(f"{k}: {x[0,i]}")

    # Return the final PageRank vector
    return x

In [7]:
# Function call to read the graph data from a file named "example.json"
G = read_graph("data/example.json")

# Print the graph data loaded from "example.json"
G

{'a': ['b', 'c', 'd'],
 'b': ['e', 'f'],
 'c': ['e'],
 'd': ['a'],
 'e': ['d'],
 'f': ['a']}

In [8]:
# Function call to compute the PageRank of the graph 'G'
# The damping factor (alpha) is set to 0.01, and the convergence threshold (epsilon) is also set to 0.01
compute_PageRank(G, 0.01, 0.01)
# note that if alpha<1 you obtain different score but the ranking remains the same

0.605
0.4900499999999998
0.3773384999999998
0.32019866999999985
0.30819121987500003
0.2789570813040001
0.2301395920758001
0.19508645545775485
0.1780982699569446
0.16049990528705663
0.13717159296642067
0.11733352554267039
0.10439872024304728
0.0932377371928985
0.08101553726442613
0.0699328313364963
0.06153219255482146
0.054527406030074374
0.047704875510938945
0.04145193210765448
0.03632099089951812
0.032017085053075885
0.028075162884862538
0.024495558456739938
0.021439364248160557
0.018840223946054654
0.016527349044354205
0.014453170072871613
0.012649993733878345
0.01109802759611938
0.009733489648710805
PageRank scores:
a: 0.31582393596234243
b: 0.10502541513974245
c: 0.10502541513974245
d: 0.2608705489588058
e: 0.1591076802661904
f: 0.05414700453317457


matrix([[0.31582394, 0.10502542, 0.10502542, 0.26087055, 0.15910768,
         0.054147  ]])